# Download Data

In [ ]:
!gdown https://drive.google.com/uc?id=1O3AMQz_CmapznZFoFDOVghTlC_uvJRNq #Download kaggle.json
!pip install kaggle #Install kaggle api
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/" #Configure the path of kaggle.json

In [ ]:
!kaggle competitions download -c comp-551-fall-2021

In [ ]:
!unzip images_l.pkl.zip
!unzip images_test.pkl.zip
!unzip images_ul.pkl.zip
!unzip labels_l.pkl.zip

In [ ]:
import pickle
import numpy as np
with open("labels_l.pkl", 'rb') as f: data = pickle.load(f)
labels_train_raw = np.array(data)
labels_train_raw = labels_train_raw.astype(np.double)
with open("images_l.pkl", 'rb') as f: data = pickle.load(f)
images_train_raw = np.array(data)
images_train_raw = images_train_raw.astype(np.double)
with open("images_ul.pkl", 'rb') as f: data = pickle.load(f)
images_unlabel = np.array(data)
with open("images_test.pkl", 'rb') as f: data = pickle.load(f)
images_test = np.array(data)
print('label for training:')
print(labels_train_raw.shape)
print('images for training:')
print(images_train_raw.shape)
print('unlabel images:')
print(images_unlabel.shape)
print('images for testing:')
print(images_test.shape)

# Data Preprocessing

In [7]:
images_train = images_train_raw.copy()
labels_digits_train = labels_train_raw[:, :10]
labels_letters_train = labels_train_raw[:, 10:]

## Without Augmentation

In [8]:
x_train = images_train[:29000,:,:].reshape(-1, 56, 56, 1)
y_train_digits = labels_digits_train[:29000,:]
y_train_letters = labels_letters_train[:29000,:]
x_val = images_train[29000:,:,:].reshape(-1, 56, 56, 1)
y_val_digits = labels_digits_train[29000:,:]
y_val_letters = labels_letters_train[29000:,:]

## With Augmentation

In [ ]:
from numpy.random import seed
from numpy.random import rand
from scipy.ndimage import rotate
import cv2
def rotation_aug_rand(threshold):
  image_size = images_train.shape[1:]
  training_size = images_train.shape[0]
  augmented = []
  for i in range(0,training_size):
    original = images_train[i]
    augmented.append(original)
    angle1 = np.random.choice([np.random.default_rng().uniform(low=-threshold, high=-threshold+15),np.random.default_rng().uniform(low=threshold-15, high=threshold)])
    rotated_img1 = rotate(original, angle=angle1, reshape=False)
    angle2 = np.random.choice([np.random.default_rng().uniform(low=-threshold, high=-threshold+15),np.random.default_rng().uniform(low=threshold-15, high=threshold)])
    rotated_img2 = rotate(original, angle=angle2, reshape=False)
    augmented.append(rotated_img1)
    augmented.append(rotated_img2)
  augmented = np.stack(augmented,axis = 0)
  return augmented

In [ ]:
augmented_train = rotation_aug_rand(36)

In [ ]:
augmented_digits_labels = np.repeat(labels_digits_train,3,axis=0)
augmented_letters_labels = np.repeat(labels_letters_train,3,axis=0)


In [ ]:
x_train = augmented_train[:80000,:,:].reshape(-1, 56, 56, 1)
y_train_digits = augmented_digits_labels[:80000,:]
y_train_letters = augmented_letters_labels[:80000,:]
x_val = augmented_train[80000:,:,:].reshape(-1, 56, 56, 1)
y_val_digits = augmented_digits_labels[80000:,:]
y_val_letters = augmented_letters_labels[80000:,:]

## Noise Reduction (Optional)
**Warning: This method may decrease performance in some cases**

In [ ]:
def noiseReduction(images_set):
  for imageIndex in range(0, images_set.shape[0]-1):
    if imageIndex % 1000 == 0:
      print(imageIndex)
    w,h = 56,56
    black_point = 0
    for x in range(0,w-1):
        for y in range(0,h-1):
            mid_pixel = images_set[imageIndex][x][y] #Find pixel value of the mid point
            # Sharpen
            if(mid_pixel>30):
              images_set[imageIndex][x][y]=255
            else:
              images_set[imageIndex][x][y]=0
            if mid_pixel > 100: # Find pixel of up, botton, left, right
                top_pixel = images_set[imageIndex][x][y-1]
                left_pixel = images_set[imageIndex][x-1][y]
                down_pixel = images_set[imageIndex][x][y+1]
                right_pixel = images_set[imageIndex][x+1][y]
    
                #Number of  black point around
                if top_pixel == 0:
                    black_point += 1
                if left_pixel == 0:
                    black_point += 1
                if down_pixel == 0:
                    black_point += 1
                if right_pixel == 0:
                    black_point += 1
                if black_point >= 3:
                    images_set[imageIndex][x][y]=0
            black_point = 0
  return images_set

In [ ]:
def noiseReduction(images_set):
  for i in range(0, images_set.shape[0]):
    images_set[i, images_set[i] < 0.25] = 0
  return images_set

In [ ]:
def minMaxNormalization(images_set):
  for i in range(0, images_set.shape[0]):
    images_set[i] = (images_set[i] - np.min(images_set[i])) / (np.max(images_set[i]) - np.min(images_set[i]))
  return images_set

# Building CNN

In [81]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Conv2D, Activation, BatchNormalization, MaxPool2D, Dropout, Flatten, Dense
from keras.models import Model
from keras.callbacks import ReduceLROnPlateau

inputs = tf.keras.layers.Input(shape=(56, 56, 1))

joint = Conv2D(64, (5, 5), padding = "same")(inputs)
joint = Activation("relu")(joint)
joint = BatchNormalization()(joint)
joint = Conv2D(64, (5, 5), padding = "same")(joint)
joint = Activation("relu")(joint)
joint = BatchNormalization()(joint)
joint = MaxPool2D((2, 2), (2, 2))(joint)
joint = Dropout(0.3)(joint)

joint = Conv2D(128, (5, 5), padding = "same")(joint)
joint = Activation("relu")(joint)
joint = BatchNormalization()(joint)
joint = Conv2D(128, (5, 5), padding = "same")(joint)
joint = Activation("relu")(joint)
joint = BatchNormalization()(joint)
joint = MaxPool2D((2, 2), (2, 2))(joint)
joint = Dropout(0.3)(joint)

joint = Conv2D(256, (5, 5), padding = "same")(joint)
joint = Activation("relu")(joint)
joint = BatchNormalization()(joint)
joint = Conv2D(256, (5, 5), padding = "same")(joint)
joint = Activation("relu")(joint)
joint = BatchNormalization()(joint)
joint = MaxPool2D((2, 2), (2, 2))(joint)
joint = Dropout(0.3)(joint)

joint = Flatten()(joint)
joint = Dense(512)(joint)
joint = Activation("relu")(joint)
joint = BatchNormalization()(joint)
joint = Dropout(0.3)(joint)

digits_branch = Dense(256)(joint)
digits_branch = Activation("relu")(digits_branch)
digits_branch = BatchNormalization()(digits_branch)
digits_branch = Dropout(0.3)(digits_branch)
digits_branch = Dense(10, activation = "softmax", name = "digits_output")(digits_branch)

letters_branch = Dense(256)(joint)
letters_branch = Activation("relu")(letters_branch)
letters_branch = BatchNormalization()(letters_branch)
letters_branch = Dropout(0.3)(letters_branch)
letters_branch = Dense(26, activation = "softmax", name = "letters_output")(letters_branch)

model = Model(inputs = inputs, outputs = [digits_branch, letters_branch])
model.compile(optimizer = "rmsprop", loss = "categorical_crossentropy", metrics=['accuracy'])

## Callbacks

In [35]:
class CombinedValidation(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    accuracy = get_combined_validation_accuracy(self.model)
    combined_val_accuracies.append(accuracy)
    print(" - combined validation accuracy: " + str(accuracy))

In [21]:
#import pickle
#model = pickle.load(open('model 98.4 98.6.pkl', 'rb'))

def get_combined_validation_accuracy(model):
  predictions = model.predict(x_val.reshape(-1, 56, 56, 1))
  predictions_indices_digits = np.argmax(predictions[0], axis = 1)
  predictions_indices_letters = np.argmax(predictions[1], axis = 1)
  validation_indices_digits = np.argmax(y_val_digits, axis = 1)
  validation_indices_letters = np.argmax(y_val_letters, axis = 1)
  correct = 0
  total = predictions_indices_digits.shape[0]
  for i in range(total):
    if predictions_indices_digits[i] == validation_indices_digits[i] and predictions_indices_letters[i] == validation_indices_letters[i]:
      correct = correct + 1
  return correct / total

#print(get_combined_validation_accuracy(model))


In [ ]:
rlrop_digits = ReduceLROnPlateau(monitor = "val_digits_output_accuracy", factor = 0.5, patience = 2, verbose = 1)
rlrop_letters = ReduceLROnPlateau(monitor = "val_letters_output_accuracy", factor = 0.5, patience = 2, verbose = 1)
combined_validation = CombinedValidation()

# Training

In [ ]:
combined_val_accuracies = []
model.fit(x = x_train, y = {"digits_output": y_train_digits, "letters_output": y_train_letters}, 
          batch_size = 128, epochs = 30, callbacks = [rlrop_digits, rlrop_letters, combined_validation],
          validation_data = (x_val, {"digits_output": y_val_digits, "letters_output": y_val_letters}),
          shuffle = True)

## With Aggregation

In [ ]:
#model aggregating without replacement

digits_list = []
letters_list = []
for i in range (0,6):
  x_train = images_train[5000 * i:5000 * (i+1),:,:].reshape(-1, 56, 56, 1)
  y_train_digits = labels_digits_train[5000 * i:5000 * (i+1),:]
  y_train_letters = labels_letters_train[5000 * i:5000 * (i+1),:]
  x_val = images_train[5000 * (5-i):5000 * (6-i),:,:].reshape(-1, 56, 56, 1)
  y_val = labels_digits_train[5000 * (5-i):5000 * (6-i),:]
  y_val_digits = labels_digits_train[5000 * (5-i):5000 * (6-i),:]
  y_val_letters = labels_letters_train[5000 * (5-i):5000 * (6-i),:]
  model.fit(x = x_train, y = {"digits_output": y_train_digits, "letters_output": y_train_letters}, 
            batch_size = 128, epochs = 30, callbacks = [rlrop_digits, rlrop_letters], 
            validation_data = (x_val, {"digits_output": y_val_digits, "letters_output": y_val_letters}),
            shuffle = True)
  predictions = model.predict(images_test.reshape(-1, 56, 56, 1))
  indices_digits = np.argmax(predictions[0], axis = 1)
  indices_letters = np.argmax(predictions[1], axis = 1)
  digits_list.append(indices_digits)
  letters_list.append(indices_letters)



In [ ]:
#aggregate the results of non-replacement
from scipy import stats
digits_predict = np.array(digits_list)
letters_predict = np.array(letters_list)

aggregated_digits = stats.mode(digits_predict,axis = 0)[0]
aggregated_letters = stats.mode(letters_predict,axis = 0)[0]

In [ ]:
#model aggregating with replacement

digits_list = []
letters_list = []
xy_train_digits =  
for i in range (0,10):
  x_train = images_train[5000 * i:5000 * (i+1),:,:].reshape(-1, 56, 56, 1)
  y_train_digits = labels_digits_train[5000 * i:5000 * (i+1),:]
  y_train_letters = labels_letters_train[5000 * i:5000 * (i+1),:]
  x_val = images_train[5000 * (5-i):5000 * (6-i),:,:].reshape(-1, 56, 56, 1)
  y_val = labels_digits_train[5000 * (5-i):5000 * (6-i),:]
  y_val_digits = labels_digits_train[5000 * (5-i):5000 * (6-i),:]
  y_val_letters = labels_letters_train[5000 * (5-i):5000 * (6-i),:]
  model.fit(x = x_train, y = {"digits_output": y_train_digits, "letters_output": y_train_letters},
            batch_size = 128, epochs = 30, callbacks = [rlrop_digits, rlrop_letters], 
            validation_data = (x_val, {"digits_output": y_val_digits, "letters_output": y_val_letters}),
            shuffle = True)
  predictions = model.predict(images_test.reshape(-1, 56, 56, 1))
  indices_digits = np.argmax(predictions[0], axis = 1)
  indices_letters = np.argmax(predictions[1], axis = 1)
  digits_list.append(indices_digits)
  letters_list.append(indices_letters)

In [ ]:
indices_digits = np.squeeze(aggregated_digits)
indices_letters = np.squeeze(aggregated_letters)

## Saving Model

In [85]:
import pickle
pickle.dump(model, open("model adam 98.2 97.9 96.9.pkl", "wb"))

INFO:tensorflow:Assets written to: ram://aceb4c23-0265-49e7-bd55-37ff0f9affff/assets


# Utilizing Unlabelled Data

In [80]:
model = pickle.load(open("model adam 98.0 98.0 96.8.pkl", "rb"))
predictions_unlabelled = model.predict(images_unlabel.reshape(-1, 56, 56, 1))
x_train_with_unlabelled = np.concatenate((x_train, images_unlabel.reshape(-1, 56, 56, 1)), axis = 0)
y_train_digits_with_unlabelled = np.concatenate((y_train_digits, predictions_unlabelled[0]), axis = 0)
y_train_letters_with_unlabelled = np.concatenate((y_train_letters, predictions_unlabelled[1]), axis = 0)

In [ ]:
combined_val_accuracies = []
model.fit(x = x_train_with_unlabelled, y = {"digits_output": y_train_digits_with_unlabelled, "letters_output": y_train_letters_with_unlabelled},
          batch_size = 128, epochs = 30, callbacks = [rlrop_digits, rlrop_letters, combined_validation],
          validation_data = (x_val, {"digits_output": y_val_digits, "letters_output": y_val_letters}),
          shuffle = True)

# Test

In [12]:
import csv
predictions = model.predict(images_test.reshape(-1, 56, 56, 1))
indices_digits = np.argmax(predictions[0], axis = 1)
indices_letters = np.argmax(predictions[1], axis = 1)
with open("predictions.csv", "w", newline="") as csvfile:
  writer = csv.writer(csvfile, delimiter=",")
  writer.writerow(["# Id", "Category"])
  for i in range(0, indices_digits.shape[0]):
    writer.writerow([i, 
                     ('0' * indices_digits[i]) + '1' + ('0' * (9 - indices_digits[i]))
                     + ('0' * indices_letters[i]) + '1' + ('0' * (25 - indices_letters[i]))
                     ])

## Bagging

In [ ]:
import pickle
import csv
import numpy as np
model1 = pickle.load(open("model adam 98.2 97.9 96.9.pkl", "rb"))
model2 = pickle.load(open("model adam 98.0 98.0 96.8.pkl", "rb"))
# model3 = pickle.load(open("model_91.9.pkl", "rb"))
# model4 = pickle.load(open("model_93.6.pkl", "rb"))
model5 = pickle.load(open("model 98.4 98.6.pkl", "rb"))
prediction1=model1.predict(images_test.reshape(-1, 56, 56, 1))
prediction2=model2.predict(images_test.reshape(-1, 56, 56, 1))
# prediction3=model3.predict(images_test.reshape(-1, 56, 56, 1))
# prediction4=model4.predict(images_test.reshape(-1, 56, 56, 1))
prediction5=model5.predict(images_test.reshape(-1, 56, 56, 1))

In [ ]:
prediction11=np.array(prediction1[0])
prediction21=np.array(prediction2[0])
# prediction31=np.array(prediction3[0])
# prediction41=np.array(prediction4[0])
prediction51=np.array(prediction5[0])
predictions1=(prediction11+prediction51)/2
prediction12=np.array(prediction1[1])
prediction22=np.array(prediction2[1])
# prediction32=np.array(prediction3[1])
# prediction42=np.array(prediction4[1])
prediction52=np.array(prediction5[1])
predictions2=(prediction12+prediction52)/2
indices_digits = np.argmax(predictions1,axis=1)
indices_letters = np.argmax(predictions2,axis=1)
with open('predictions1127_4.csv', 'w', newline='') as csvfile:
  writer = csv.writer(csvfile, delimiter=',')
  writer.writerow(['# Id', 'Category'])
  for i in range(0, 15000):
    writer.writerow([i, 
                     ('0' * indices_digits[i]) + '1'  + ('0' * (9 - indices_digits[i]))
                     + ('0' * indices_letters[i]) + '1' + ('0' * (25 - indices_letters[i]))
                     ])

# Plotting

In [ ]:
import matplotlib.pyplot as plt
combined_val_accuracies_2 = combined_val_accuracies.copy()
print(combined_val_accuracies_2)
plt.rcParams["figure.figsize"] = (4,2)
plt.plot(range(1, 31), combined_val_accuracies_1)
plt.plot(range(1, 31), combined_val_accuracies_2)
plt.legend(["Without unlabelled data", "With unlabelled data fed back"])
plt.xlabel("Epoch")
plt.ylabel("Validation Accuracy")
plt.show()